In [1]:
import pandas as pd

orders_df = pd.read_excel('Orders 1.xlsx')
visits_df = pd.read_excel('Visits 1.xlsx')

print("Orders Data:")
print(orders_df.head())

print("\nVisits Data:")
print(visits_df.head())


Orders Data:
   ORDER_ID          CREATED_AT           NMV  SALES_STATUS_ID  \
0   5275311 2023-07-28 06:30:42  15824.500000                6   
1   5211037 2023-07-18 04:57:33   9436.000000                6   
2   5483299 2023-08-27 03:55:31  11766.083333                6   
3   5691171 2023-09-26 04:23:49  10018.500000                6   
4   5738195 2023-10-04 00:23:16   4982.500000                6   

            RETAILER_ID        day  
0  JlkbkCvjnCFHchhKDGLz 2023-07-28  
1  JlkbkCvjnCFHchhKDGLz 2023-07-18  
2  JlkbkCvjnCFHchhKDGLz 2023-08-27  
3  JlkbkCvjnCFHchhKDGLz 2023-09-26  
4  JlkbkCvjnCFHchhKDGLz 2023-10-04  

Visits Data:
   VISIT_ID          CREATED_AT  AGENT_ID           RETAILER_ID        Day
0   2606224 2023-09-14 14:11:57       631  qfEP18Ergf5J5rlRHZD0 2023-09-14
1   2606328 2023-09-14 14:31:23       631  r7LdHSMZ7TwEhdEcyhgq 2023-09-14
2   2606385 2023-09-14 14:40:58       631  qEFmNkcjtOYq05XraREp 2023-09-14
3   2606470 2023-09-14 14:55:05       631  yuByoSyVWnz

In [3]:
# Convert 'Created at' columns to datetime
orders_df['CREATED_AT'] = pd.to_datetime(orders_df['CREATED_AT'])
visits_df['CREATED_AT'] = pd.to_datetime(visits_df['CREATED_AT'])

# Check for missing values
print(orders_df.isnull().sum())
print(visits_df.isnull().sum())


ORDER_ID           0
CREATED_AT         0
NMV                0
SALES_STATUS_ID    0
RETAILER_ID        0
day                0
dtype: int64
VISIT_ID       0
CREATED_AT     0
AGENT_ID       0
RETAILER_ID    0
Day            0
dtype: int64


In [8]:
#Question 1: Orders Placed by Each Agent
# Merge visits and orders on Retailer ID and Date
merged_df = pd.merge(visits_df, orders_df, on=['RETAILER_ID'], suffixes=('_visit', '_order'))

# Filter orders that happened on the same day as visits
same_day_orders = merged_df[merged_df['CREATED_AT_visit'].dt.date == merged_df['CREATED_AT_order'].dt.date]

# Calculate the number of orders per agent
orders_per_agent = same_day_orders.groupby('AGENT_ID')['ORDER_ID'].nunique().reset_index()
orders_per_agent.columns = ['AGENT_ID', 'Total_Orders']
print(orders_per_agent)


    AGENT_ID  Total_Orders
0        631             5
1        732            14
2        752           674
3        777            11
4        782           168
..       ...           ...
56      3239            27
57      3265             6
58      3266            48
59      3271            11
60      3274            15

[61 rows x 2 columns]


In [10]:
#Question 2: Strike Rate Per Agent
visits_per_agent = visits_df.groupby('AGENT_ID')['VISIT_ID'].nunique().reset_index()
visits_per_agent.columns = ['AGENT_ID', 'Total_Visits']

strike_rate = pd.merge(orders_per_agent, visits_per_agent, on='AGENT_ID')
strike_rate['Strike_Rate'] = strike_rate['Total_Orders'] / strike_rate['Total_Visits']
print(strike_rate)


    AGENT_ID  Total_Orders  Total_Visits  Strike_Rate
0        631             5             9     0.555556
1        732            14            18     0.777778
2        752           674          1549     0.435119
3        777            11            22     0.500000
4        782           168           444     0.378378
..       ...           ...           ...          ...
56      3239            27            47     0.574468
57      3265             6            52     0.115385
58      3266            48           137     0.350365
59      3271            11            44     0.250000
60      3274            15            29     0.517241

[61 rows x 4 columns]


In [11]:
#Question 3: Average Ticket Size Per Agent

ticket_size_per_agent = same_day_orders.groupby('AGENT_ID')['NMV'].mean().reset_index()
ticket_size_per_agent.columns = ['AGENT_ID', 'Average_Ticket_Size']
print(ticket_size_per_agent)


    AGENT_ID  Average_Ticket_Size
0        631          3024.950000
1        732          1185.250000
2        752          1727.040931
3        777          1517.090909
4        782          2572.177370
..       ...                  ...
56      3239          1547.324074
57      3265          1642.791667
58      3266          1657.439796
59      3271          2201.090909
60      3274          1277.116667

[61 rows x 2 columns]


In [12]:
#Question 4: Organic vs. Inorganic Activations

first_orders = orders_df.groupby('RETAILER_ID')['CREATED_AT'].min().reset_index()
first_orders.columns = ['RETAILER_ID', 'First_Order_Date']

inorganic = pd.merge(first_orders, visits_df, left_on=['RETAILER_ID', 'First_Order_Date'], right_on=['RETAILER_ID', 'CREATED_AT'])

organic = first_orders[~first_orders['RETAILER_ID'].isin(inorganic['RETAILER_ID'])]

organic_count = organic['First_Order_Date'].dt.date.value_counts().reset_index()
organic_count.columns = ['Date', 'Organic_Activations']

inorganic_count = inorganic['First_Order_Date'].dt.date.value_counts().reset_index()
inorganic_count.columns = ['Date', 'Inorganic_Activations']

print(organic_count)
print(inorganic_count)


           Date  Organic_Activations
0    2023-07-03                  804
1    2023-07-04                  768
2    2023-07-02                  750
3    2023-07-01                  697
4    2023-07-05                  594
..          ...                  ...
99   2023-08-11                   11
100  2023-09-01                   11
101  2023-10-06                    9
102  2023-09-15                    9
103  2023-10-05                    6

[104 rows x 2 columns]
         Date  Inorganic_Activations
0  2023-09-16                      1
1  2023-07-10                      1
2  2023-07-25                      1
3  2023-07-19                      1


In [15]:
#Question 5: Calculate Retention Rate Per Month

orders_df['Order_Month'] = orders_df['CREATED_AT'].dt.to_period('M')

monthly_orders = orders_df.groupby(['RETAILER_ID', 'Order_Month']).size().reset_index(name='Order_Count')

monthly_orders['Next_Month'] = monthly_orders['Order_Month'] + 1
retained = pd.merge(monthly_orders, monthly_orders, left_on=['RETAILER_ID', 'Next_Month'], right_on=['RETAILER_ID', 'Order_Month'])
retained = retained.groupby('Order_Month_x').size().reset_index(name='Retained_Retailers')
retained['Order_Month'] = retained['Order_Month_x'].astype(str)
retained = retained.drop(columns=['Order_Month_x'])

total_retailers = monthly_orders.groupby('Order_Month')['RETAILER_ID'].nunique().reset_index(name='Total_Retailers')
total_retailers['Order_Month'] = total_retailers['Order_Month'].astype(str)

retention = pd.merge(total_retailers, retained, on='Order_Month', how='left').fillna(0)
retention['Retention_Rate'] = (retention['Retained_Retailers'] / retention['Total_Retailers']) * 100

print(retention)


  Order_Month  Total_Retailers  Retained_Retailers  Retention_Rate
0     2023-07             8711              7050.0       80.932155
1     2023-08             8731              6963.0       79.750315
2     2023-09             8574              4375.0       51.026359
3     2023-10             5003                 0.0        0.000000


In [16]:
#question 6 : Segment Retailers Based on NMV

total_nmv = orders_df.groupby('RETAILER_ID')['NMV'].sum().reset_index()

low_threshold = 1000
high_threshold = 5000

total_nmv['Segment'] = pd.cut(total_nmv['NMV'], bins=[-1, low_threshold, high_threshold, float('inf')], 
                               labels=['Low', 'Medium', 'High'])
print(total_nmv)


                RETAILER_ID       NMV Segment
0      00HQ6W34DdXawhOi8uoG  16498.75    High
1      018X9rzASd56jVUGTQEL  20982.75    High
2      01zQ3UJEsVJwhOOgC04B   1210.00  Medium
3      0218m8gj8hI5LVesP4v7  25181.61    High
4      02c246Mp7um3Bm8u4iAC   1585.80  Medium
...                     ...       ...     ...
11592  zyMtSJoB7R0aR9KOtnHA  17262.51    High
11593  zz6ugf6iQiEtCneMHEsb   2930.75  Medium
11594  zzLurtu5ZbWufIMS5z0x  18845.25    High
11595  zzjkocIGyYh8vH87YKkv   9505.25    High
11596  zzoEOUbaI3PH4MroJmch   3626.00  Medium

[11597 rows x 3 columns]


In [17]:
#question 7: Calculate Time Between Visits Per Agent

visits_df = visits_df.sort_values(by=['AGENT_ID', 'CREATED_AT'])

visits_df['Time_Diff'] = visits_df.groupby('AGENT_ID')['CREATED_AT'].diff().dt.days

avg_time_between_visits = visits_df.groupby('AGENT_ID')['Time_Diff'].mean().reset_index()
avg_time_between_visits.columns = ['AGENT_ID', 'Avg_Days_Between_Visits']

print(avg_time_between_visits)



    AGENT_ID  Avg_Days_Between_Visits
0        631                 1.250000
1        732                 2.411765
2        752                 0.022610
3        777                 0.047619
4        778                 0.000000
..       ...                      ...
61      3239                 0.021739
62      3265                 0.039216
63      3266                 0.014706
64      3271                 0.000000
65      3274                 0.035714

[66 rows x 2 columns]


In [18]:
#question 8: Identify Anomalies or Unusual Patterns
from scipy import stats

orders_df['NMV_zscore'] = stats.zscore(orders_df['NMV'])
outliers = orders_df[orders_df['NMV_zscore'].abs() > 3]  # Consider z-scores > 3 as outliers

print("Potential_Anomalies:")
print(outliers)



Potential_Anomalies:
       ORDER_ID          CREATED_AT       NMV  SALES_STATUS_ID  \
0       5275311 2023-07-28 06:30:42  15824.50                6   
223     5682794 2023-09-25 00:08:08  49012.50                6   
224     5689837 2023-09-25 23:06:52  16639.50                6   
225     5288780 2023-07-30 17:17:13  16947.50                6   
227     5261563 2023-07-26 01:43:24  15588.00                6   
...         ...                 ...       ...              ...   
88728   5487855 2023-08-27 20:37:29  17291.50                7   
89075   5656772 2023-09-20 21:39:30  43500.00                6   
89111   5390585 2023-08-14 00:49:25  21627.00                6   
89114   5437240 2023-08-20 16:51:26  30830.00               12   
89146   5523958 2023-09-02 14:47:47  21258.75                6   

                RETAILER_ID        day Order_Month  NMV_zscore  
0      JlkbkCvjnCFHchhKDGLz 2023-07-28     2023-07    3.228872  
223    9JfELs5zmjQNwh7rQADt 2023-09-25     2023-09   11.

In [20]:
#question 9 : Top Agents Encouraging Future Orders

future_orders = orders_df[orders_df['CREATED_AT'] > orders_df.groupby('RETAILER_ID')['CREATED_AT'].transform('min')]
future_orders_count = future_orders[future_orders['RETAILER_ID'].isin(visits_df['RETAILER_ID'])].groupby('RETAILER_ID').size().reset_index(name='Future_Orders_Count')
top_agents = pd.merge(visits_df[['RETAILER_ID', 'AGENT_ID']].drop_duplicates(), future_orders_count, on='RETAILER_ID')
top_agents = top_agents.groupby('AGENT_ID')['Future_Orders_Count'].sum().reset_index().sort_values(by='Future_Orders_Count', ascending=False)

print("Top_Agents_Encouraging_Future_Orders:")
print(top_agents)



Top_Agents_Encouraging_Future_Orders:
    AGENT_ID  Future_Orders_Count
29      2352                 2875
6        805                 2667
16      1168                 2631
37      2914                 2125
26      1754                 2104
..       ...                  ...
7        848                   19
4        778                   16
27      1837                    7
22      1405                    3
52      3219                    1

[63 rows x 2 columns]


In [22]:
#question 10 : Develop a Metric to Assess Agents’ Performance
agent_performance = pd.merge(strike_rate, ticket_size_per_agent, on='AGENT_ID')
agent_performance = pd.merge(agent_performance, avg_time_between_visits, on='AGENT_ID')

agent_performance['Performance_Score'] = (
    agent_performance['Strike_Rate'] * 0.4 +
    agent_performance['Average_Ticket_Size'] * 0.3 +
    (1 / agent_performance['Avg_Days_Between_Visits']) * 0.3 )

agent_performance = agent_performance.sort_values(by='Performance_Score', ascending=False)

print("Agent_Performance:")
print(agent_performance)


Agent_Performance:
    AGENT_ID  Total_Orders  Total_Visits  Strike_Rate  Average_Ticket_Size  \
29      2787             3            53     0.056604          1029.416667   
59      3271            11            44     0.250000          2201.090909   
54      3237             3            19     0.157895          1391.750000   
48      3211            18            50     0.360000          1411.842105   
46      3203             1            12     0.083333           935.000000   
..       ...           ...           ...          ...                  ...   
20      1390            48           123     0.390244          1231.753698   
60      3274            15            29     0.517241          1277.116667   
1        732            14            18     0.777778          1185.250000   
25      1837             2             1     2.000000          4495.000000   
26      1892             1             1     1.000000          3045.000000   

    Avg_Days_Between_Visits  Performance_Sco